This is an updated version of a Blog posts at http://www.pythonocc.org/quick-examples/machine-learning-and-building-models/ that is unfortunately no longer online

![](img/ml01.png)

In the post I will outline a recent research project in which we looked into Machine Learning for the classification and validation of IFC models (Krijnen & Tamke, 2015). A copy of the full paper can be found here: http://link.springer.com/chapter/10.1007%2F978-3-319-24208-8_33

It is common practice to use automation for finding design errors in building models. Proprietary applications exist that hardcode such design rules into fixed program code. These rules are then explicitly stated and match the legislation and norms that a building has to comply to. However:

- Formalizing these requirements into computationally decidable logic can be a daunting task
- Furthermore, within these definitions rules that relate to common sense (e.g. there should be enough room for a door to be opened) are typically not defined

Machine Learning can complement these imperative design validation procedures as it enables automated processes to gather common constellations of building elements and flag situations that deviate from what is typically constructed. Note that such situations are then not necessarily wrong, but they could be an indication to simply have another look.

In this particular case we predominantly look into geometrical descriptors for elements individually. Elements of that same type that are different from the norm can then be found. The code example below, shows the relative ease of accomplishing something similar by relying on modules such as IfcOpenShell and pythonOCC. It leads to the discovery of a set of elements that should not have been characterized as walls, but rather are beams. Such kinds of misclassification errors might seem trivial, but can cause havoc as rules apply to specific classes of elements and disciplines operate on the elements that match their expertise. Therefore in this case, the structural engineer might overlook these elements if they have been misclassified as architectural walls.

In [ ]:
import operator

import OCC.Core.GProp
import OCC.Core.BRepGProp

import ifcopenshell
import ifcopenshell.geom

import numpy

# RGBA colors for the visualisation of elements
RED, GRAY = (1,0,0,1), (0.6, 0.6, 0.6, 0.1)

ifc_file = ifcopenshell.open("models/Duplex_A_20110907_optimized.ifc")

# Settings to specify usage of pyOCC
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)

# Some helper functions to map to the list of walls
def create_shape(elem):
    return ifcopenshell.geom.create_shape(settings, elem)

def calc_volume(s):
    props = OCC.Core.GProp.GProp_GProps()
    OCC.Core.BRepGProp.brepgprop_VolumeProperties(s.geometry, props)
    return props.Mass()
    
def calc_area(s):
    props = OCC.Core.GProp.GProp_GProps()
    OCC.Core.BRepGProp.brepgprop_SurfaceProperties(s.geometry, props)
    return props.Mass()
    
def normalize(li):
    mean, std = numpy.mean(li), numpy.std(li)
    return map(lambda v: abs(v-mean) / std, li)

# Obtain a list of walls from the model
walls = ifc_file.by_type("IfcWall")
# Create geometry for these walls
shapes = list(map(create_shape, walls))
# Calculate their volumes
volumes = list(map(calc_volume, shapes))
# Calculate their surface areas
areas = list(map(calc_area, shapes))
# Compose a feature from the two measures
feature = normalize(list(map(operator.truediv, areas, volumes)))

# Initialize the viewer
from ifc_viewer import ifc_viewer
        
viewer = ifc_viewer()

# Loop over the sorted pairs of feature
# values and corresponding geometry
for d, _, s in sorted(zip(feature, range(len(walls)), shapes)):
    c = RED if d > 1. else GRAY
    viewer.DisplayShape(s.geometry, shape_color=c[0:3])

viewer.Display()